In [1]:
import pandas as pd
import numpy as np
import json
import warnings

warnings.filterwarnings("ignore")

from collections import defaultdict, Counter
from datetime import datetime

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import STL

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
FILE = "consolidated_file_cleaned_v2.csv"

df = pd.read_csv(FILE)

# Clean and prepare
df["time"] = pd.to_datetime(df["time"])
df = df.sort_values("time")
df = df[df["suitable_for_seasonal_analysis"] == True]
df["month"] = df["time"].dt.to_period("M").dt.to_timestamp()

# Monthly aggregation: top-level + second-level category pairs
category_data = (
    df.groupby(["month", "top-level_category", "second-level_category"])["sold/m"]
      .sum()
      .reset_index()
)

# Create paired category column (for output compatibility)
category_data["category_pair"] = (
    category_data["top-level_category"] + " > " + category_data["second-level_category"]
)

print(f"✓ Data loaded: {len(df)} records")
print(f"  Categories: {category_data['category_pair'].nunique()} unique category pairs")
print(f"  Date range: {df['time'].min().date()} to {df['time'].max().date()}")
print("\nSample data:")
print(category_data.head(10))

✓ Data loaded: 7142715 records
  Categories: 264 unique category pairs
  Date range: 2022-03-01 to 2025-11-01

Sample data:
       month top-level_category              second-level_category    sold/m  \
0 2022-03-01              Audio                Amplifiers & Mixers       0.0   
1 2022-03-01              Audio  Audio & Video Cables & Converters    8994.0   
2 2022-03-01              Audio   Earphones, Headphones & Headsets  518302.0   
3 2022-03-01              Audio              Home Audio & Speakers   45449.0   
4 2022-03-01              Audio                      Media Players       0.0   
5 2022-03-01              Audio                        Microphones   24925.0   
6 2022-03-01              Audio                             Others       0.0   
7 2022-03-01        Automobiles    Automobile Exterior Accessories   68102.0   
8 2022-03-01        Automobiles    Automobile Interior Accessories   83516.0   
9 2022-03-01        Automobiles             Automobile Spare Parts   68145.0

In [3]:
SEASON_JSON = "common_categories.json"

def norm(s):
    return s.strip().lower()

# Load season mapping from JSON
common_category_to_seasons = defaultdict(set)
common_subcategories = {}

with open(SEASON_JSON, "r", encoding="utf-8") as f:
    season_map_raw = json.load(f)

# Parse JSON structure to map subcategories to seasons
for season, items in season_map_raw.items():
    for item in items:
        if isinstance(item, dict):
            for parent_cat, subcats in item.items():
                for subcat in subcats:
                    # Store normalized mapping
                    common_category_to_seasons[norm(subcat)].add(season)
                    common_subcategories[norm(subcat)] = (parent_cat, subcat, season)

common_keys = set(common_category_to_seasons.keys())

print(f"✓ Loaded {len(season_map_raw)} seasons from common_categories.json")
print(f"  Total common subcategories: {len(common_subcategories)}")
print("\nSample common categories:")
for i, (cat, seasons) in enumerate(list(common_category_to_seasons.items())[:5]):
    print(f"  {cat} → {seasons}")

✓ Loaded 7 seasons from common_categories.json
  Total common subcategories: 118

Sample common categories:
  hoodies & sweatshirts → {'Rainy Season'}
  jackets, coats & vests → {'Rainy Season', 'Christmas Season'}
  pants → {'Back-to-School Season', 'Rainy Season'}
  sweaters & cardigans → {'Rainy Season'}
  sleepwear → {'Rainy Season'}


In [4]:
def is_common_subcategory(subcat_name):
    """Check if a subcategory is in the common list"""
    n = norm(subcat_name)
    
    # Exact match first
    if n in common_category_to_seasons:
        return True, common_category_to_seasons[n]
    
    # Fuzzy substring match
    matched_seasons = set()
    for ck in common_keys:
        if ck in n or n in ck:
            matched_seasons |= common_category_to_seasons[ck]
    
    if matched_seasons:
        return True, matched_seasons
    
    return False, set()

def detect_uncommon_high_demand(category_data):
    """
    Identify uncommon subcategories that show strong seasonal patterns
    Returns dict: {subcategory: (top_level, demand_score)}
    """
    uncommon_high_demand = {}
    
    for subcat in category_data["second-level_category"].unique():
        # Skip if it's a known common category
        is_common, _ = is_common_subcategory(subcat)
        if is_common:
            continue
        
        # Get data for this subcategory
        subcat_data = category_data[category_data["second-level_category"] == subcat]
        ts = subcat_data.set_index("month")["sold/m"]
        
        # Need at least 18 months of data
        if len(ts) < 18 or ts.sum() == 0:
            continue
        
        # Calculate seasonal strength using STL decomposition
        try:
            stl = STL(ts, period=12, robust=True).fit()
            seasonal_strength = 1 - (stl.resid.var() / (stl.seasonal + stl.resid).var())
            
            # Also consider total demand
            total_demand = ts.sum()
            demand_score = seasonal_strength * np.log1p(total_demand)
            
            # Threshold for uncommon but in-demand
            if seasonal_strength >= 0.35:  # Shows seasonality
                top_level = subcat_data["top-level_category"].iloc[0]
                uncommon_high_demand[subcat] = {
                    "top_level": top_level,
                    "seasonal_strength": round(seasonal_strength, 3),
                    "total_demand": round(total_demand, 2),
                    "demand_score": round(demand_score, 3)
                }
        except:
            continue
    
    return uncommon_high_demand

# Identify uncommon categories
uncommon_categories = detect_uncommon_high_demand(category_data)

print(f"\n✓ Identified {len(uncommon_categories)} uncommon but high-demand categories:")
for subcat, info in sorted(uncommon_categories.items(), key=lambda x: x[1]['demand_score'], reverse=True)[:10]:
    print(f"  • {subcat}")
    print(f"    - Top Level: {info['top_level']}")
    print(f"    - Seasonal Strength: {info['seasonal_strength']}")
    print(f"    - Total Demand: {info['total_demand']}")


✓ Identified 15 uncommon but high-demand categories:
  • Girl Shoes
    - Top Level: Baby & Kids Fashion
    - Seasonal Strength: 0.744
    - Total Demand: 4002635.0
  • Boy Shoes
    - Top Level: Baby & Kids Fashion
    - Seasonal Strength: 0.679
    - Total Demand: 2648488.0
  • Desktop & Laptop Components
    - Top Level: Computers & Accessories
    - Seasonal Strength: 0.62
    - Total Demand: 1623034.0
  • Services
    - Top Level: Tickets, Vouchers & Services
    - Seasonal Strength: 0.777
    - Total Demand: 68137.0
  • Furniture
    - Top Level: Home & Living
    - Seasonal Strength: 0.44
    - Total Demand: 26606259.0
  • Boy Clothes
    - Top Level: Baby & Kids Fashion
    - Seasonal Strength: 0.364
    - Total Demand: 19453782.0
  • Large Household Appliances
    - Top Level: Home Appliances
    - Seasonal Strength: 0.366
    - Total Demand: 15092347.0
  • Letters & Envelopes
    - Top Level: Stationery
    - Seasonal Strength: 0.386
    - Total Demand: 5957026.0
  • CD, DV

In [5]:
def safe_sarima_fit(ts):
    """Try multiple SARIMA configurations safely"""
    sarima_orders = [
        ((1,1,1), (1,1,1,12)),
        ((0,1,1), (1,1,1,12)),
        ((1,1,0), (0,1,1,12)),
        ((0,1,1), (0,1,1,12)),
        ((1,0,0), (1,0,0,12)),
    ]
    
    for order, sorder in sarima_orders:
        try:
            model = SARIMAX(ts, order=order, seasonal_order=sorder,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            return model.fit(disp=False)
        except:
            continue
    
    return None

def train_sarima_lstm_hybrid(ts, cat_name, forecast_steps=12):
    """
    Train SARIMA + LSTM on residuals for hybrid forecasting
    Returns: (forecasts, metrics_dict)
    """
    if len(ts) < 18:
        return None, None
    
    try:
        # Step 1: Fit SARIMA
        sarima_fit = safe_sarima_fit(ts)
        if sarima_fit is None:
            return None, None
        
        sarima_fc = sarima_fit.forecast(steps=forecast_steps)
        residuals = sarima_fit.resid
        
        # Step 2: Prepare residuals for LSTM
        scaler = MinMaxScaler()
        scaled_res = scaler.fit_transform(residuals.values.reshape(-1, 1))
        
        # Create sequences for LSTM
        seq_len = 12
        X_train, y_train = [], []
        
        for i in range(seq_len, len(scaled_res)):
            X_train.append(scaled_res[i-seq_len:i, 0])
            y_train.append(scaled_res[i, 0])
        
        if len(X_train) < 5:
            # Not enough data for LSTM, use SARIMA only
            return sarima_fc.values, {"model_type": "SARIMA_only"}
        
        X_train = np.array(X_train).reshape(-1, seq_len, 1)
        y_train = np.array(y_train)
        
        # Step 3: Train LSTM on residuals
        lstm = Sequential([
            LSTM(32, activation="tanh", return_sequences=True, input_shape=(seq_len, 1)),
            LSTM(16, activation="tanh"),
            Dense(1)
        ])
        
        lstm.compile(optimizer="adam", loss="mse")
        es = EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)
        
        history = lstm.fit(X_train, y_train, epochs=50, batch_size=8, 
                          verbose=0, callbacks=[es])
        
        # Step 4: Forecast residuals
        last_seq = scaled_res[-seq_len:].copy()
        residual_preds = []
        seq = last_seq.copy()
        
        for _ in range(forecast_steps):
            pred = lstm.predict(seq.reshape(1, seq_len, 1), verbose=0)[0, 0]
            residual_preds.append(pred)
            seq = np.append(seq[1:], pred).reshape(-1, 1)
        
        residual_fc = scaler.inverse_transform(np.array(residual_preds).reshape(-1, 1)).flatten()
        
        # Step 5: Combine SARIMA + LSTM residuals
        hybrid_fc = sarima_fc.values + residual_fc
        hybrid_fc = np.maximum(hybrid_fc, 0)  # Ensure non-negative
        
        metrics = {
            "model_type": "SARIMA_LSTM_Hybrid",
            "lstm_epochs_trained": len(history.history['loss']),
            "final_lstm_loss": round(float(history.history['loss'][-1]), 6)
        }
        
        return hybrid_fc, metrics
        
    except Exception as e:
        print(f"  ⚠ Error training {cat_name}: {str(e)}")
        return None, None

print("✓ SARIMA-LSTM training function defined")

✓ SARIMA-LSTM training function defined


In [6]:
print("\n" + "="*80)
print("TRAINING SARIMA-LSTM HYBRID MODELS FOR ALL CATEGORIES")
print("="*80)

# Combine common + uncommon categories
all_subcategories = category_data["second-level_category"].unique()

forecast_results = []
model_performance = []
FORECAST_STEPS = 12

for idx, subcat in enumerate(all_subcategories):
    # Check if common or uncommon
    is_common, seasons = is_common_subcategory(subcat)
    
    if is_common:
        category_type = "COMMON"
        category_seasons = list(seasons)
    else:
        category_type = "UNCOMMON"
        category_seasons = []  # Will be inferred from peaks
    
    # Get time series
    subcat_data = category_data[category_data["second-level_category"] == subcat]
    ts = subcat_data.set_index("month")["sold/m"]
    
    if len(ts) < 18:
        continue
    
    top_level = subcat_data["top-level_category"].iloc[0]
    
    # Train hybrid model
    print(f"\n[{idx+1}/{len(all_subcategories)}] {subcat} ({category_type})")
    
    forecasts, metrics = train_sarima_lstm_hybrid(ts, subcat, forecast_steps=FORECAST_STEPS)
    
    if forecasts is None:
        print(f"  ✗ Training failed")
        continue
    
    print(f"  ✓ Trained ({metrics['model_type']})")
    
    # Generate future dates
    future_dates = pd.date_range(
        start=ts.index.max() + pd.offsets.MonthBegin(),
        periods=FORECAST_STEPS,
        freq="MS"
    )
    
    # Store forecasts
    for d, fc in zip(future_dates, forecasts):
        forecast_results.append({
            "top_level_category": top_level,
            "second_level_category": subcat,
            "category_type": category_type,
            "forecast_date": d.strftime("%Y-%m"),
            "predicted_demand": round(max(fc, 0), 2),
            "seasons": "|".join(category_seasons) if category_seasons else "TBD"
        })
    
    # Store model performance
    model_performance.append({
        "second_level_category": subcat,
        "category_type": category_type,
        "top_level_category": top_level,
        **metrics
    })

forecast_df = pd.DataFrame(forecast_results)
performance_df = pd.DataFrame(model_performance)

print(f"\n✓ Training complete!")
print(f"  Forecasts generated: {len(forecast_df)}")
print(f"  Models trained: {len(performance_df)}")

print("\nForecast Sample:")
print(forecast_df.head(15))


TRAINING SARIMA-LSTM HYBRID MODELS FOR ALL CATEGORIES

[1/215] Amplifiers & Mixers (UNCOMMON)


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


  ✓ Trained (SARIMA_LSTM_Hybrid)

[2/215] Audio & Video Cables & Converters (COMMON)


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


  ✓ Trained (SARIMA_LSTM_Hybrid)

[3/215] Earphones, Headphones & Headsets (COMMON)


...

  ✓ Trained (SARIMA_LSTM_Hybrid)

[214/215] Heels (UNCOMMON)


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


  ✓ Trained (SARIMA_LSTM_Hybrid)

[215/215] Wedges (COMMON)


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


  ✓ Trained (SARIMA_LSTM_Hybrid)

✓ Training complete!
  Forecasts generated: 2580
  Models trained: 215

Forecast Sample:
   top_level_category              second_level_category category_type  \
0               Audio                Amplifiers & Mixers      UNCOMMON   
1               Audio                Amplifiers & Mixers      UNCOMMON   
2               Audio                Amplifiers & Mixers      UNCOMMON   
3               Audio                Amplifiers & Mixers      UNCOMMON   
4               Audio                Amplifiers & Mixers      UNCOMMON   
5               Audio                Amplifiers & Mixers      UNCOMMON   
6               Audio                Amplifiers & Mixers      UNCOMMON   
7               Audio                Amplifiers & Mixers      UNCOMMON   
8               Audio                Amplifiers & Mixers      UNCOMMON   
9               Audio                Amplifiers & Mixers      UNCOMMON   
10              Audio                Amplifiers & Mixers      U

In [7]:
print("\n" + "="*80)
print("INFERRING PEAK SEASONS FOR UNCOMMON CATEGORIES")
print("="*80)

# Define season mapping (aligned with frontend)
season_month_mapping = {
    'Rainy Season': ['06','07','08','09','10','11'],
    'Back-to-School Season': ['06','07','08','09'],
    'Halloween / Undas Season': ['10','11'],
    'Christmas Season': ['09','10','11','12'],
    "Valentine's Season": ['02'],
    'Holy Week / Lent Season': ['03','04'],
    'Summer Season': ['03','04','05'],
}

# Reverse mapping: month -> seasons
month_to_seasons = {}
for season, months in season_month_mapping.items():
    for month in months:
        if month not in month_to_seasons:
            month_to_seasons[month] = []
        month_to_seasons[month].append(season)

# Group forecasts by category and identify peak months
uncommon_df = forecast_df[forecast_df["category_type"] == "UNCOMMON"].copy()

category_peaks = {}

for subcat in uncommon_df["second_level_category"].unique():
    subcat_fc = uncommon_df[uncommon_df["second_level_category"] == subcat].copy()
    
    # Get month numbers from forecast dates (as strings with leading zeros)
    subcat_fc["forecast_month"] = pd.to_datetime(subcat_fc["forecast_date"]).dt.month.astype(str).str.zfill(2)
    
    # Average demand by month
    monthly_demand = subcat_fc.groupby("forecast_month")["predicted_demand"].mean()
    
    # Find top 3 peak months
    top_months = monthly_demand.nlargest(3).index.tolist()
    
    # Map months to all applicable seasons
    inferred_seasons = []
    for month in top_months:
        if month in month_to_seasons:
            inferred_seasons.extend(month_to_seasons[month])
    
    # Remove duplicates while preserving order
    inferred_seasons = list(dict.fromkeys(inferred_seasons))
    
    category_peaks[subcat] = {
        "top_level": uncommon_df[uncommon_df["second_level_category"] == subcat]["top_level_category"].iloc[0],
        "peak_months": top_months,
        "inferred_seasons": inferred_seasons,
        "demand_by_month": monthly_demand.to_dict()  # Show all months for reference
    }

print(f"✓ Peak seasons inferred for {len(category_peaks)} uncommon categories\n")

# Display detailed results
print("Top 10 Uncommon Categories with Inferred Peak Seasons:")
print("-" * 80)

for subcat, info in list(category_peaks.items())[:10]:
    print(f"\n  📦 {subcat}")
    print(f"     Top Level: {info['top_level']}")
    print(f"     Peak Months: {', '.join(info['peak_months'])}")
    print(f"     Associated Seasons: {', '.join(info['inferred_seasons'])}")
    print(f"     Monthly Demand: {info['demand_by_month']}")


INFERRING PEAK SEASONS FOR UNCOMMON CATEGORIES
✓ Peak seasons inferred for 73 uncommon categories

Top 10 Uncommon Categories with Inferred Peak Seasons:
--------------------------------------------------------------------------------

  📦 Amplifiers & Mixers
     Top Level: Audio
     Peak Months: 01, 02, 03
     Associated Seasons: Valentine's Season, Holy Week / Lent Season, Summer Season
     Monthly Demand: {'01': 0.0, '02': 0.0, '03': 0.0, '04': 0.0, '05': 0.0, '06': 0.0, '07': 0.0, '08': 0.0, '09': 0.0, '10': 0.0, '11': 0.0, '12': 0.0}

  📦 Media Players
     Top Level: Audio
     Peak Months: 09, 11, 05
     Associated Seasons: Rainy Season, Back-to-School Season, Christmas Season, Halloween / Undas Season, Summer Season
     Monthly Demand: {'01': 1122.46, '02': 1596.42, '03': 1598.33, '04': 4106.06, '05': 5703.78, '06': 1933.88, '07': 3091.15, '08': 2199.03, '09': 25048.69, '10': 4008.54, '11': 8627.07, '12': 3573.7}

  📦 Microphones
     Top Level: Audio
     Peak Months: 1

In [8]:
print("\n" + "="*80)
print("CREATING COMBINED SEASON-TO-CATEGORY MAPPING")
print("="*80)

# Build complete mapping: seasons → (common + uncommon categories)
season_category_mapping = defaultdict(list)

# Add common categories
for season, items in season_map_raw.items():
    for item in items:
        if isinstance(item, dict):
            for parent_cat, subcats in item.items():
                for subcat in subcats:
                    season_category_mapping[season].append({
                        "type": "common",
                        "second_level": subcat,
                        "top_level": parent_cat
                    })

# Add uncommon categories based on inferred peaks
for subcat, info in category_peaks.items():
    for season in info["inferred_seasons"]:
        if season in season_category_mapping:  # Only add to recognized seasons
            season_category_mapping[season].append({
                "type": "uncommon",
                "second_level": subcat,
                "top_level": info["top_level"],
                "confidence": "inferred_from_demand"
            })

# Remove duplicates
for season in season_category_mapping:
    seen = set()
    unique_items = []
    for item in season_category_mapping[season]:
        key = item["second_level"]
        if key not in seen:
            seen.add(key)
            unique_items.append(item)
    season_category_mapping[season] = unique_items

# Export as JSON for support system
output_json = {
    "metadata": {
        "generated_at": datetime.now().isoformat(),
        "forecast_months": FORECAST_STEPS,
        "common_categories_count": sum(1 for s in season_category_mapping.values() for item in s if item["type"] == "common"),
        "uncommon_categories_count": sum(1 for s in season_category_mapping.values() for item in s if item["type"] == "uncommon")
    },
    "seasons": dict(season_category_mapping)
}

# Save to JSON
output_file = "season_category_demand_mapping.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_json, f, indent=2, ensure_ascii=False)

print(f"✓ Mapping saved to {output_file}\n")

# Display summary
print("Season-to-Category Mapping Summary:")
for season in sorted(season_category_mapping.keys()):
    items = season_category_mapping[season]
    common_count = sum(1 for item in items if item["type"] == "common")
    uncommon_count = sum(1 for item in items if item["type"] == "uncommon")
    print(f"\n  {season}:")
    print(f"    Common Categories: {common_count}")
    print(f"    Uncommon Categories: {uncommon_count}")
    print(f"    Total: {len(items)}")


CREATING COMBINED SEASON-TO-CATEGORY MAPPING
✓ Mapping saved to season_category_demand_mapping.json

Season-to-Category Mapping Summary:

  Back-to-School Season:
    Common Categories: 22
    Uncommon Categories: 62
    Total: 84

  Christmas Season:
    Common Categories: 36
    Uncommon Categories: 57
    Total: 93

  Halloween / Undas Season:
    Common Categories: 13
    Uncommon Categories: 35
    Total: 48

  Holy Week / Lent Season:
    Common Categories: 13
    Uncommon Categories: 17
    Total: 30

  Rainy Season:
    Common Categories: 31
    Uncommon Categories: 68
    Total: 99

  Summer Season:
    Common Categories: 30
    Uncommon Categories: 38
    Total: 68

  Valentine's Season:
    Common Categories: 22
    Uncommon Categories: 10
    Total: 32


In [9]:
print("\n" + "="*80)
print("EXPORTING DATA FOR SUPPORT SYSTEM")
print("="*80)

# Save forecasts
forecast_df.to_csv("hybrid_sarima_lstm_all_categories_forecast.csv", index=False)
print(f"✓ Saved forecasts to 'hybrid_sarima_lstm_all_categories_forecast.csv'")

# Save model performance
performance_df.to_csv("model_performance_sarima_lstm.csv", index=False)
print(f"✓ Saved model performance to 'model_performance_sarima_lstm.csv'")

# Summary statistics
print(f"\n📊 Forecast Summary:")
print(f"  Total records: {len(forecast_df)}")
print(f"  Common categories: {len(forecast_df[forecast_df['category_type']=='COMMON']['second_level_category'].unique())}")
print(f"  Uncommon categories: {len(forecast_df[forecast_df['category_type']=='UNCOMMON']['second_level_category'].unique())}")
print(f"  Date range: {forecast_df['forecast_date'].min()} to {forecast_df['forecast_date'].max()}")

print(f"\n✓ All files exported successfully!")


EXPORTING DATA FOR SUPPORT SYSTEM
✓ Saved forecasts to 'hybrid_sarima_lstm_all_categories_forecast.csv'
✓ Saved model performance to 'model_performance_sarima_lstm.csv'

📊 Forecast Summary:
  Total records: 2580
  Common categories: 142
  Uncommon categories: 73
  Date range: 2025-12 to 2026-11

✓ All files exported successfully!


In [11]:
print("\n" + "="*80)
print("PROCESSING FORECAST DATA: DEDUPLICATION & SEASON SEPARATION")
print("="*80)

# Read the generated forecast CSV
forecast_df_raw = pd.read_csv("hybrid_sarima_lstm_all_categories_forecast.csv")

print(f"\nOriginal forecast data: {len(forecast_df_raw)} rows")

# Step 1: Deduplicate based on top_level_category, second_level_category, category_type
# (ignoring forecast_date and predicted_demand)
dedup_df = forecast_df_raw.drop_duplicates(
    subset=['top_level_category', 'second_level_category', 'category_type'],
    keep='first'
)

print(f"After deduplication: {len(dedup_df)} rows")

# Step 2: Separate seasons - if multiple seasons exist (separated by |), create separate rows
final_results = []

for idx, row in dedup_df.iterrows():
    top_level = row['top_level_category']
    second_level = row['second_level_category']
    category_type = row['category_type']
    seasons_str = row['seasons']
    
    # Check if there are multiple seasons
    if seasons_str == 'TBD' or pd.isna(seasons_str):
        # Single row for TBD categories
        final_results.append({
            'top_level_category': top_level,
            'second_level_category': second_level,
            'category_type': category_type,
            'season': 'Others'
        })
    else:
        # Split seasons and create a row for each season
        seasons_list = [s.strip() for s in str(seasons_str).split('|')]
        
        for season in seasons_list:
            final_results.append({
                'top_level_category': top_level,
                'second_level_category': second_level,
                'category_type': category_type,
                'season': season
            })

# Create final dataframe
final_df = pd.DataFrame(final_results)

# Remove any remaining duplicates (in case a category maps to same season multiple times)
final_df = final_df.drop_duplicates(
    subset=['top_level_category', 'second_level_category', 'category_type', 'season'],
    keep='first'
).reset_index(drop=True)

print(f"After season separation: {len(final_df)} rows")

# Save to CSV
output_file = "hybrid_sarima_lstm_categories_unique_with_seasons.csv"
final_df.to_csv(output_file, index=False)

print(f"\n✓ Processed forecast saved to '{output_file}'")

# Display summary
print("\n📊 Summary of Unique Category-Season Mapping:")
print("-" * 80)

summary = final_df.groupby('season').agg({
    'category_type': lambda x: (x == 'COMMON').sum(),
    'second_level_category': 'count'
}).rename(columns={
    'category_type': 'Common Count',
    'second_level_category': 'Total Count'
})
summary['Uncommon Count'] = summary['Total Count'] - summary['Common Count']
summary = summary[['Total Count', 'Common Count', 'Uncommon Count']]

print(summary)

print("\n📋 Sample Data (first 20 rows):")
print(final_df.head(20).to_string(index=False))

print(f"\n✓ Processing complete!")



PROCESSING FORECAST DATA: DEDUPLICATION & SEASON SEPARATION

Original forecast data: 2580 rows
After deduplication: 215 rows
After season separation: 292 rows

✓ Processed forecast saved to 'hybrid_sarima_lstm_categories_unique_with_seasons.csv'

📊 Summary of Unique Category-Season Mapping:
--------------------------------------------------------------------------------
                          Total Count  Common Count  Uncommon Count
season                                                             
Back-to-School Season              41            41               0
Christmas Season                   41            41               0
Halloween / Undas Season           15            15               0
Holy Week / Lent Season            14            14               0
Others                             73             0              73
Rainy Season                       32            32               0
Summer Season                      52            52               0
Valentine's Se